## Setup
Install accompanying pinon library. It is better to install in dev mode in the conda environment with `pip install -e .` Jupyter server must restart to see changes to conda environment.
Auto reload the pinon module.  Other imports may be ignored for auto reload e.g.  `%aimport -simfin`. Note use of option 2 for auto reload and excluding other modules, rather than option 1 and including only reloadable modules.  This is because option 1 does not work correctly with aliased imports.

In [ ]:
# !pip install -e ..

%load_ext autoreload
%autoreload 2
%aimport -pandas
%aimport -simfin
%aimport -os

%aimport pinon

%matplotlib inline

# imports
import pandas as pd
import seaborn as sns
import os

import pinon as pn
import pinon.names as pn_cols
import simfin as sf
import simfin.names as sf_cols  # column name shortcuts

In [ ]:
config = pn.Config('master', 'V', 10)

In [12]:
c = pn.Comps(config)
c.run()
c.comp_ratios

KeyboardInterrupt: 

In [ ]:
c.calc_fair_value()